### Prerequisite Packages

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from torcheval.metrics import BinaryPrecision, BinaryRecall, BinaryF1Score
from sklearn.model_selection import train_test_split, KFold

In [2]:
sys.path.append('../')

from modules.cross_attentionb import CrossAttentionB
from modules.dataloader import load_npy_files
from modules.classifier import DenseLayer, BCELoss, CustomLoss, BCEWithLogits
from modules.linear_transformation import LinearTransformations

### Data Loading

In [3]:
class MultimodalDataset(Dataset):
    def __init__(self, id_label_df, text_features, audio_features, video_features):
        self.id_label_df = id_label_df
        
        # Convert feature lists to dictionaries for fast lookup
        self.text_features = {os.path.basename(file).split('.')[0]: tensor for file, tensor in text_features}
        self.audio_features = {os.path.basename(file).split('_')[1].split('.')[0]: tensor for file, tensor in audio_features}
        self.video_features = {os.path.basename(file).split('_')[0]: tensor for file, tensor in video_features}

        # List to store missing files
        self.missing_files = []

        # Filter out entries with missing files
        self.valid_files = self._filter_valid_files()

    def _filter_valid_files(self):
        valid_indices = []
        missing_files = []

        for idx in range(len(self.id_label_df)):
            imdbid = self.id_label_df.iloc[idx]['IMDBid']

            # Check if the IMDBid exists in each modality's features
            if imdbid in self.text_features and imdbid in self.audio_features and imdbid in self.video_features:
                valid_indices.append(idx)
            else:
                missing_files.append({'IMDBid': imdbid})

        # Filter id_label_df to only include valid rows
        self.id_label_df = self.id_label_df.iloc[valid_indices].reset_index(drop=True)
        self.missing_files = missing_files
        
        # Update valid_indices to reflect the new indices after resetting
        valid_indices = list(range(len(self.id_label_df)))

        # Return valid indices
        return valid_indices

    def __len__(self):
        return len(self.valid_files)
    

    def __getitem__(self, idx):
        

        # Get the original index from the filtered valid files
        original_idx = self.valid_files[idx]
        imdbid = self.id_label_df.iloc[original_idx]['IMDBid']
        label = self.id_label_df.iloc[original_idx]['Label']

        # Retrieve data from the loaded features
        text_data = self.text_features.get(imdbid, torch.zeros((1024,)))
        audio_data = self.audio_features.get(imdbid, torch.zeros((1, 197, 768)))
        video_data = self.video_features.get(imdbid, torch.zeros((95, 768)))
        
        # Define label mapping
        label_map = {'red': 1, 'green': 0} 
        
        # Convert labels to tensor using label_map
        try:
            label_data = torch.tensor([label_map[label]], dtype=torch.float32)
        except KeyError as e:
            print(f"Error: Label '{e}' not found in label_map.")
            raise

        return imdbid, text_data, audio_data, video_data, label_data


In [4]:
def collate_fn(batch):
    # Unpack batch elements
    imdbids, text_data, audio_data, video_data, label_data = zip(*batch)

    # Convert lists to tensors
    text_data = torch.stack(text_data)
    audio_data = torch.stack(audio_data)

    # Padding for video data
    # Determine maximum length of video sequences in the batch
    video_lengths = [v.size(0) for v in video_data]
    max_length = max(video_lengths)

    # Pad video sequences to the maximum length
    video_data_padded = torch.stack([
        F.pad(v, (0, 0, 0, max_length - v.size(0)), "constant", 0)
        for v in video_data
    ])

    # Convert labels to tensor and ensure the shape [batch_size, 1]
    label_data = torch.stack(label_data)  # Convert list of tensors to a single tensor

    return imdbids, text_data, audio_data, video_data_padded, label_data


In [5]:
# Load the labels DataFrame
id_label_df = pd.read_excel('../misc/MM-Trailer_dataset.xlsx')

# Define the directories
text_features_dir = 'C:\\Users\\edjin\\OneDrive\\Documents\\Programming Files\\Thesis\\SMCA\\misc\\textStream_BERT\\feature_vectors\\feature_vectors'
audio_features_dir = 'C:\\Users\\edjin\\OneDrive\\Documents\\Programming Files\\Thesis\\SMCA\\misc\\audio_fe\\logmel_spectrograms'
video_features_dir = 'C:\\Users\\edjin\\OneDrive\\Documents\\Programming Files\\Thesis\\SMCA\\misc\\visualStream_ViT\\feature_vectors'

# Load the feature vectors from each directory
text_features = load_npy_files(text_features_dir)
audio_features = load_npy_files(audio_features_dir)
video_features = load_npy_files(video_features_dir)

print(f"Number of text feature vectors loaded: {len(text_features)}")
print(f"Number of audio feature vectors loaded: {len(audio_features)}")
print(f"Number of video feature vectors loaded: {len(video_features)}")

# Drop unnecessary columns
id_label_df = id_label_df.drop(columns=['Movie Title', 'URL'])
print("ID Label Shape:", id_label_df.shape)

full_dataset = MultimodalDataset(id_label_df, text_features, audio_features, video_features)
# full_dataset.__len__()

print("Length of id_label_df:", len(full_dataset.id_label_df))
print("Length in valid_files:", len(full_dataset.valid_files))

print("Max index in id_label_df:", full_dataset.id_label_df.index.max())
print("Max index in valid_files:", max(full_dataset.valid_files))

for item in full_dataset.missing_files:
                print(f"IMDBid: {item['IMDBid']}")
print(f"Total IMDB IDs with missing files: {len(full_dataset.missing_files)}")

# First, filter the id_label_df using the valid indices before creating dataset splits
filtered_id_label_df = id_label_df.iloc[full_dataset.valid_files].reset_index(drop=True)

# perform train-test split on the filtered DataFrame
train_df, val_test_df = train_test_split(
    filtered_id_label_df, test_size=0.3, random_state=42, stratify=filtered_id_label_df['Label'])

# Further splitting remaining set into validation and test sets
val_df, test_df = train_test_split(
    val_test_df, test_size=0.5, random_state=42, stratify=val_test_df['Label'])

print(train_df.shape)
print(val_df.shape)
print(test_df.shape)

print("Train label distribution:", train_df['Label'].value_counts())
print("Validation label distribution:", val_df['Label'].value_counts())
print("Test label distribution:", test_df['Label'].value_counts())

print("-" * 40)

# create datasets based on these splits
train_dataset = MultimodalDataset(train_df, text_features, audio_features, video_features)
val_dataset = MultimodalDataset(val_df, text_features, audio_features, video_features)
test_dataset = MultimodalDataset(test_df, text_features, audio_features, video_features)

# Calculate weights for the classes in the training set
class_counts = train_df['Label'].value_counts().to_dict()
class_weights = {label: 1.0 / count for label, count in class_counts.items()}
sample_weights = [class_weights[label] for label in train_df['Label']]

# Initialize the WeightedRandomSampler
weighted_sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),
    replacement=True
)

# RESAMPLING
# train_dataloader = DataLoader(train_dataset, batch_size=32, sampler=weighted_sampler, num_workers=0, collate_fn=collate_fn)

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=True, num_workers=0, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=0, collate_fn=collate_fn)


Number of text feature vectors loaded: 1353
Number of audio feature vectors loaded: 1353
Number of video feature vectors loaded: 1353
ID Label Shape: (1432, 2)
Length of id_label_df: 1353
Length in valid_files: 1353
Max index in id_label_df: 1352
Max index in valid_files: 1352
IMDBid: tt0988595
IMDBid: tt1724962
IMDBid: tt0758730
IMDBid: tt0200469
IMDBid: tt0389790
IMDBid: tt4503510
IMDBid: tt0401997
IMDBid: tt1349482
IMDBid: tt0082186
IMDBid: tt1094661
IMDBid: tt0924129
IMDBid: tt2437712
IMDBid: tt2917484
IMDBid: tt3053228
IMDBid: tt0099371
IMDBid: tt2101341
IMDBid: tt0099385
IMDBid: tt0259288
IMDBid: tt1336621
IMDBid: tt1087842
IMDBid: tt2937696
IMDBid: tt1288558
IMDBid: tt2524674
IMDBid: tt4158096
IMDBid: tt3675748
IMDBid: tt1800302
IMDBid: tt1104733
IMDBid: tt0465494
IMDBid: tt0498353
IMDBid: tt0110093
IMDBid: tt5580036
IMDBid: tt5962210
IMDBid: tt2180411
IMDBid: tt0405052
IMDBid: tt2713642
IMDBid: tt1772925
IMDBid: tt2980472
IMDBid: tt0988083
IMDBid: tt2935564
IMDBid: tt0140336
IM

In [6]:
# Iterate through DataLoader
for imdbids, text_data, audio_data, video_data, labels in train_dataloader:
    print("IMDB IDs:", imdbids)
    print("Text Data:", text_data)
    print("Audio Data:", audio_data)
    print("Video Data:", video_data)
    print("Labels:", labels)
    break  # Just to demonstrate the first batch

IMDB IDs: ('tt0765476', 'tt0113965', 'tt0117420', 'tt2294629', 'tt0397313', 'tt3882074', 'tt1046947', 'tt0141369', 'tt0270197', 'tt2380564', 'tt1073241', 'tt0454776', 'tt0286306', 'tt0489270', 'tt0322589', 'tt0890870')
Text Data: tensor([[ 0.3638, -0.4746, -0.6777,  ..., -0.6358, -0.5378,  0.4044],
        [ 0.6473, -0.6516, -1.1244,  ..., -0.5755, -0.3885,  0.5081],
        [ 0.2702, -0.6794, -1.0013,  ..., -1.2219, -0.4856,  0.2977],
        ...,
        [-0.2448,  0.2331, -1.3465,  ..., -0.5651, -0.4837,  0.2202],
        [ 0.6542, -0.4119, -0.6177,  ..., -0.9829, -0.4346,  0.3270],
        [-0.6718, -0.4077, -0.9236,  ..., -1.2391, -0.6866, -0.1162]])
Audio Data: tensor([[[[-0.2960, -0.1368,  0.0091,  ...,  0.1739,  0.1066,  0.0488],
          [-0.2288, -0.2299, -0.0661,  ...,  0.1294,  0.1059, -0.0929],
          [-0.3299, -0.0048,  0.0386,  ...,  0.1776,  0.0359, -0.1124],
          ...,
          [-0.2374,  0.0926, -0.0129,  ...,  0.1501,  0.0333, -0.0069],
          [-0.2327,  

### Important Functions

In [7]:
# Cross Attention Function
def PairCrossAttention(modalityAlpha, modalityBeta, d_out_kq=768, d_out_v=768):
    cross_attn = CrossAttentionB(modalityAlpha.shape[-1], modalityBeta.shape[-1], d_out_kq, d_out_v)
    modalityAlphaBeta = cross_attn(modalityAlpha, modalityBeta)
    return modalityAlphaBeta

In [8]:
class MultiheadCrossAttention(nn.Module):
    def __init__(self, embed_dim, num_heads=8):
        super().__init__()
        self.multihead_attn = nn.MultiheadAttention(embed_dim, num_heads, batch_first=True)
        
    def forward(self, query, key_value):
        # Ensure inputs are 3D: (batch_size, sequence_length, embed_dim)
        if query.dim() == 2:
            query = query.unsqueeze(1)  # Add sequence length dimension
        if key_value.dim() == 2:
            key_value = key_value.unsqueeze(1)  # Add sequence length dimension
            
        output, _ = self.multihead_attn(query, key_value, key_value)
        return output

### Hyperparameters and Important Assignments

In [9]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

# !!! Choose if pytorch's multiheadattention or own crossattention
isMultiHead = True 

# Modality assignments
modality_assignments = {
    'modalityAlpha': 'audio_features',
    'modalityBeta': 'text_features',
    'modalityGamma': 'video_features'
}

# Define the loss function
isBCELoss = True                          # !!! SET ACCORDINGLY !!!
criterion = BCELoss()
# criterion = BCEWithLogits()
# criterion = CustomLoss(pos_weight=2.94)  

# Hyperparameters
threshold = 0.5        # for predictions
learning_rate = 1e-5
dropout_rate = 0.4    # for FinalClassifier
num_epochs = 10       # Set the number of epochs you want to train for

# !!! Choose Classifier !!! False = Dense Layer, True = Final Classifier
isFinalClassifier = True

Device: cuda


### SMCA Functions and Model

In [10]:
def SMCAStage1(modalityAlpha, modalityBeta, d_out_kq, d_out_v, device):
    
    if isMultiHead: 
        cross_attn = MultiheadCrossAttention(d_out_v).to(device)
    else:
        cross_attn = CrossAttentionB(modalityAlpha.shape[-1], modalityBeta.shape[-1], d_out_kq, d_out_v).to(device)

    # Cross-attention: Alpha -> Beta
    alphaBeta = cross_attn(modalityAlpha, modalityBeta)  # Shape: (batch_size, num_queries, d_out_v)

    # Cross-attention: Beta -> Alpha
    betaAlpha = cross_attn(modalityBeta, modalityAlpha)  # Shape: (batch_size, num_kv, d_out_v)

    # Get the sequence lengths
    seq_len_alpha = alphaBeta.size(1)  # This is num_queries
    seq_len_beta = betaAlpha.size(1)    # This is num_kv

    # Instead of expanding, use padding or trimming
    max_seq_len = max(seq_len_alpha, seq_len_beta)

    # Ensure both alphaBeta and betaAlpha are of shape (batch_size, max_seq_len, d_out_v)
    if seq_len_alpha < max_seq_len:
        alphaBeta = torch.nn.functional.pad(alphaBeta, (0, 0, 0, max_seq_len - seq_len_alpha), value=0)

    if seq_len_beta < max_seq_len:
        betaAlpha = torch.nn.functional.pad(betaAlpha, (0, 0, 0, max_seq_len - seq_len_beta), value=0)

    # Concatenate cross-attention outputs along the feature dimension (-1)
    modalityAlphaBeta = torch.cat((alphaBeta, betaAlpha), dim=-1)  # Shape: (batch_size, max_seq_len, 2 * d_out_v)

    return modalityAlphaBeta


In [11]:
class ProjectionLayer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ProjectionLayer, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.linear(x)

In [12]:
def SMCAStage2(modalityAlphaBeta, modalityGamma, d_out_kq, d_out_v, device):
    # modalityAlphaBeta: (batch_size, seq_len, 2 * d_out_v) [output of Stage 1]
    
    # Initialize the projection layer for modalityAlphaBeta
    projection_layer = ProjectionLayer(modalityAlphaBeta.shape[-1], d_out_v).to(device)

    # Project modalityAlphaBeta to (batch_size, seq_len, d_out_v)
    modalityAlphaBetaProjected = projection_layer(modalityAlphaBeta)

    # Initialize the cross-attention module
    if isMultiHead: 
        cross_attn = MultiheadCrossAttention(d_out_v).to(device)
    else:
        cross_attn = CrossAttentionB(modalityAlphaBetaProjected.shape[-1], modalityGamma.shape[-1], d_out_kq, d_out_v).to(device)

    # Cross-attention: AlphaBeta -> Gamma
    alphaBetaGamma = cross_attn(modalityAlphaBetaProjected, modalityGamma)  # Shape: (batch_size, seq_len_alphaBeta, d_out_v)

    # Cross-attention: Gamma -> AlphaBeta
    gammaAlphaBeta = cross_attn(modalityGamma, modalityAlphaBetaProjected)  # Shape: (batch_size, seq_len_gamma, d_out_v)

    # Get the sequence lengths for both modalities
    seq_len_alphaBeta = alphaBetaGamma.size(1)
    seq_len_gamma = gammaAlphaBeta.size(1)

    # Pad the smaller sequence to match the larger one (expanding to before)
    max_seq_len = max(seq_len_alphaBeta, seq_len_gamma)

    if seq_len_alphaBeta < max_seq_len:
        alphaBetaGamma = torch.nn.functional.pad(alphaBetaGamma, (0, 0, 0, max_seq_len - seq_len_alphaBeta), value=0)

    if seq_len_gamma < max_seq_len:
        gammaAlphaBeta = torch.nn.functional.pad(gammaAlphaBeta, (0, 0, 0, max_seq_len - seq_len_gamma), value=0)

    # Concatenate along the feature dimension (-1)
    multimodal_representation = torch.cat((alphaBetaGamma, gammaAlphaBeta), dim=-1)  # Shape: (batch_size, max(seq_len_alphaBeta, seq_len_gamma), 2 * d_out_v)

    # Apply Global Average Pooling across the feature (sequence to before)
    GAP = torch.mean(multimodal_representation, dim=1)  # Shape: (batch_size, 2 * d_out_v)

    return GAP

In [13]:
class SMCAModel(nn.Module):
    def __init__(self, d_out_kq, d_out_v, device):
        super(SMCAModel, self).__init__()
        self.d_out_kq = d_out_kq
        self.d_out_v = d_out_v
        self.device = device
    
    def forward(self, modalityAlpha, modalityBeta, modalityGamma):
        # Stage 1: Cross attention between modalityAlpha and modalityBeta
        modalityAlphaBeta = SMCAStage1(modalityAlpha, modalityBeta, self.d_out_kq, self.d_out_v, self.device)

        # Stage 2: Cross attention with modalityAlphaBeta (as query) and modalityGamma (as key-value)
        multimodal_representation = SMCAStage2(modalityAlphaBeta, modalityGamma, self.d_out_kq, self.d_out_v, self.device)

        return multimodal_representation

In [14]:
def train_model(model, dense_layer, dataloader, criterion, optimizer, device):
    model.train()
    dense_layer.train()
    total_loss = 0.0

    for imdbids, text_features, audio_features, video_features, targets in dataloader:
        text_features, audio_features, video_features, targets = (
            text_features.to(device),
            audio_features.to(device),
            video_features.to(device),
            targets.to(device).view(-1)
        )
                
        # Squeeze the audio features to remove the extra dimension
        audio_features = audio_features.squeeze(1).to(device) 

        # Apply linear transformations to match dimensions
        linear_transform_text = LinearTransformations(text_features.shape[-1], 768).to(device)   

        # Transform features to match the target dimension
        text_features = linear_transform_text(text_features).to(device)    
        
        transformed_features = {
            'audio_features': audio_features,
            'text_features': text_features,
            'video_features': video_features
        }

        outputs = model(
            modalityAlpha=transformed_features[modality_assignments['modalityAlpha']].to(device),  # Use the dictionary for modality assignment
            modalityBeta=transformed_features[modality_assignments['modalityBeta']].to(device),
            modalityGamma=transformed_features[modality_assignments['modalityGamma']].to(device)
        )

        # Pass the fused features through the dense layer
        predictions = dense_layer(outputs).view(-1)

        # Compute loss
        loss = criterion(predictions, targets)
        total_loss += loss.item()
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return total_loss / len(dataloader)

In [15]:
def evaluate_model(model, dense_layer, dataloader, criterion, device):
    model.eval()
    dense_layer.eval()
    total_loss = 0.0

    # Initialize the metrics for binary classification
    precision_metric = BinaryPrecision().to(device)
    recall_metric = BinaryRecall().to(device)
    f1_metric = BinaryF1Score().to(device)

    precision_metric.reset()
    recall_metric.reset()
    f1_metric.reset()

    all_predictions = []
    all_targets = []
    
    with torch.no_grad():
         for imdbids, text_features, audio_features, video_features, targets in dataloader:
            text_features, audio_features, video_features, targets = (
                text_features.to(device),
                audio_features.to(device),
                video_features.to(device),
                targets.to(device).view(-1)
            )
        
            # Squeeze the audio features to remove the extra dimension
            audio_features = audio_features.squeeze(1).to(device) 

            # Apply linear transformations to match dimensions
            linear_transform_text = LinearTransformations(text_features.shape[-1], 768).to(device)   

            # Transform features to match the target dimension
            text_features = linear_transform_text(text_features).to(device)    
            
            transformed_features = {
                'audio_features': audio_features.to(device),
                'text_features': text_features.to(device),
                'video_features': video_features.to(device)
            }

            outputs = model(
                modalityAlpha=transformed_features[modality_assignments['modalityAlpha']].to(device),  # Use the dictionary for modality assignment
                modalityBeta=transformed_features[modality_assignments['modalityBeta']].to(device),
                modalityGamma=transformed_features[modality_assignments['modalityGamma']].to(device)
            )

            # Pass the fused features through the dense layer
            predictions = dense_layer(outputs).view(-1)

            all_predictions.extend(predictions.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
            
            # Compute loss
            loss = criterion(predictions, targets)
            total_loss += loss.item()

            # !!!Apply if BCEWithLogits or CustomLoss!!!
            if not isBCELoss:
                predictions = torch.sigmoid(predictions)

            # Apply threshold to get binary predictions
            preds = (predictions >= threshold).float()
            
            # Print model predictions and targets for each batch
            print("-" * 15, "Eval", "-" * 15)
            print(f"Movie Trailer IDs:\n {imdbids}")
            print(f"Predictions (raw):\n  {np.round(predictions.cpu().numpy(), 3)}")            
            print(f"Binary Predictions:\n {preds.cpu().numpy()}")
            print(f"Targets:\n            {targets.cpu().numpy()}")
            

            # Update the precision, recall, and F1 score metrics
            precision_metric.update(preds.long(), targets.long())
            recall_metric.update(preds.long(), targets.long())
            f1_metric.update(preds.long(), targets.long())

    # Compute precision, recall, and F1 score
    precision = precision_metric.compute().item()
    recall = recall_metric.compute().item()
    f1_score = f1_metric.compute().item()

    average_loss = total_loss / len(dataloader)
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1_score:.4f}")
    
    return average_loss, precision, recall, f1_score

In [16]:
def test_model(model, dense_layer, dataloader, criterion, device):
    model.eval()
    dense_layer.eval()
    total_loss = 0

    # Initialize the metrics for binary classification
    precision_metric = BinaryPrecision().to(device)
    recall_metric = BinaryRecall().to(device)
    f1_metric = BinaryF1Score().to(device)

    with torch.no_grad():
        for imdbids, text_features, audio_features, video_features, targets in dataloader:
            text_features, audio_features, video_features, targets = (
                text_features.to(device),
                audio_features.to(device),
                video_features.to(device),
                targets.to(device).view(-1)
            )
            
            # Squeeze the audio features to remove the extra dimension
            audio_features = audio_features.squeeze(1).to(device) 

            # Apply linear transformations to match dimensions
            linear_transform_text = LinearTransformations(text_features.shape[-1], 768).to(device)   

            # Transform features to match the target dimension
            text_features = linear_transform_text(text_features).to(device)    
            
            transformed_features = {
                'audio_features': audio_features.to(device),
                'text_features': text_features.to(device),
                'video_features': video_features.to(device)
            }

            outputs = model(
                modalityAlpha=transformed_features[modality_assignments['modalityAlpha']].to(device),  # Use the dictionary for modality assignment
                modalityBeta=transformed_features[modality_assignments['modalityBeta']].to(device),
                modalityGamma=transformed_features[modality_assignments['modalityGamma']].to(device)
            )

            # Pass the fused features through the dense layer
            predictions = dense_layer(outputs).view(-1)
                
            # Compute loss
            loss = criterion(predictions, targets)
            total_loss += loss.item()

            # !!!Apply if BCEWithLogits or CustomLoss!!!
            if not isBCELoss:
                predictions = torch.sigmoid(predictions)

            # Apply threshold to get binary predictions
            preds = (predictions >= threshold).float()

            # Print model predictions and targets for each batch
            print("-" * 15, "Test", "-" * 15)
            print(f"Movie Trailer IDs:\n {imdbids}")
            print(f"Predictions (raw):  {np.round(predictions.cpu().numpy(), 3)}")            
            print(f"Binary Predictions: {preds.cpu().numpy()}")
            print(f"Targets:            {targets.cpu().numpy()}")
            
            # Update the precision, recall, and F1 score metrics
            precision_metric.update(preds.long(), targets.long())
            recall_metric.update(preds.long(), targets.long())
            f1_metric.update(preds.long(), targets.long())

    # Compute precision, recall, and F1 score
    precision = precision_metric.compute().item()
    recall = recall_metric.compute().item()
    f1_score = f1_metric.compute().item()

    average_loss = total_loss / len(dataloader)

    print(f"Test Precision: {precision:.4f}")
    print(f"Test Recall: {recall:.4f}")
    print(f"Test F1 Score: {f1_score:.4f}")
    print(f"Test Loss: {average_loss:.4f}")

    return average_loss, precision, recall, f1_score


In [17]:
def get_optimizer(parameters, lr=learning_rate):
    # Create an optimizer, for example, Adam
    return optim.Adam(parameters, lr=lr)

In [18]:
class FinalClassifier(nn.Module):
    def __init__(self, input_size, dropout_rate=dropout_rate):
        super(FinalClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)  # First fully connected layer
        self.bn1 = nn.BatchNorm1d(256)          # Batch normalization after first layer
        self.fc2 = nn.Linear(256, 128)          # Second fully connected layer
        self.bn2 = nn.BatchNorm1d(128)          # Batch normalization after second layer
        # self.fc3 = nn.Linear(256, 128)          # Third fully connected layer
        # self.bn3 = nn.BatchNorm1d(128)          # Batch normalization after third layer
        # self.fc4 = nn.Linear(128, 64)          # Second fully connected layer
        # self.bn4 = nn.BatchNorm1d(64)          # Batch normalization after third layer

        self.dropout = nn.Dropout(dropout_rate) # Dropout layer
        self.dense = nn.Linear(128, 1)          # Final dense layer for binary classification
        self.relu = nn.ReLU()                    # ReLU activation
        self.sigmoid = nn.Sigmoid()              # Sigmoid activation for final output

    def forward(self, x):
        x = self.fc1(x)                         # First fully connected layer
        x = self.bn1(x)                         # Apply batch normalization
        x = self.relu(x)                        # Apply ReLU activation

        x = self.fc2(x)                         # second fully connected layer
        x = self.bn2(x)                         # Apply batch normalization
        x = self.relu(x)                        # Apply ReLU activation
        x = self.dropout(x)                     # Apply dropout
        
        # x = self.fc3(x)                         # third fully connected layer
        # x = self.bn3(x)                         # Apply batch normalization
        # x = self.relu(x)                        # Apply ReLU activation

        # x = self.fc4(x)                         # fourth fully connected layer
        # x = self.bn4(x)                         # Apply batch normalization
        # x = self.relu(x)                        # Apply ReLU activation
        # x = self.dropout(x)                     # Apply dropout
        
        x = self.dense(x)                       # Final dense layer
        if isBCELoss:
            x = self.sigmoid(x)                  # Apply sigmoid activation
        return x                                 # Output probabilities for BCELoss


In [19]:
if __name__ == "__main__":
    torch.manual_seed(42)
    
    # Determine the output dimensions
    output_dim = 768

    # Initialize the SMCA model A
    model = SMCAModel(output_dim, output_dim, device).to(device)  # Dimension for d_out_kq and d_out_v

    

    # Own DenseLayer or FinalClassifier
    if isFinalClassifier:
        dense_layer = FinalClassifier(output_dim*2).to(device) 
    else:
        dense_layer = DenseLayer(output_dim*2).to(device)

    optimizer = get_optimizer(list(model.parameters()) + list(dense_layer.parameters()), learning_rate)

    for epoch in range(num_epochs):
        print("-" * 40)
        print(f"Epoch {epoch + 1}/{num_epochs}")

        # Ensure you have a dataloader that yields inputs and targets
        train_loss = train_model(model=model, dense_layer=dense_layer, dataloader=train_dataloader, criterion=criterion, optimizer=optimizer, device=device)

        for name, param in model.named_parameters():
            if param.grad is None:
                print("After train: model:", "No gradient for:", name)
        
        for name, param in dense_layer.named_parameters():
            if param.grad is None:
                print("After train: classifier:", "No gradient for:", name)

        # Validate step
        val_loss, precision, recall, f1_score = evaluate_model(model=model, dense_layer=dense_layer, dataloader=val_dataloader, criterion=criterion, device=device)

        print(f"Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

    # Testing the model
    print("-" * 40)
    print("Testing the model on the test set...")
    test_loss, test_precision, test_recall, test_f1_score = test_model(model=model, dense_layer=dense_layer, dataloader=test_dataloader, criterion=criterion, device=device)


----------------------------------------
Epoch 1/10
--------------- Eval ---------------
Movie Trailer IDs:
 ('tt1564349', 'tt0854678', 'tt0432291', 'tt0472562', 'tt0810913', 'tt0477051', 'tt1205558', 'tt2059171', 'tt0211181', 'tt0116722', 'tt0377091', 'tt0332452', 'tt4560436', 'tt0208092', 'tt1547230', 'tt1027747')
Predictions (raw):
  [0.464 0.484 0.448 0.461 0.491 0.477 0.449 0.465 0.464 0.463 0.475 0.453
 0.487 0.472 0.445 0.472]
Binary Predictions:
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Targets:
            [0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0.]
--------------- Eval ---------------
Movie Trailer IDs:
 ('tt0119906', 'tt0473464', 'tt2926810', 'tt0467197', 'tt4663548', 'tt3829266', 'tt0993846', 'tt0209958', 'tt0472043', 'tt4103724', 'tt0021814', 'tt0106500', 'tt2278388', 'tt1956620', 'tt0465234', 'tt0188453')
Predictions (raw):
  [0.493 0.478 0.49  0.461 0.501 0.456 0.503 0.462 0.451 0.523 0.462 0.507
 0.51  0.491 0.494 0.466]
Binary Predictions:
 [0. 0. 0. 0. 

### K-Fold Evaluation

In [20]:
def cross_validate_model(
    dataset, 
    model_class, 
    dense_layer_class, 
    num_folds, 
    num_epochs, 
    batch_size, 
    learning_rate,
    output_file,
    device=None
):
    # Set device configuration
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}")
    
    # Ensure the output directory exists
    output_dir = os.path.dirname(output_file)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)  # Creates the directory if it does not exist
    
    # Determine the output dimensions
    output_dim = 768

    # Initialize the KFold splitter
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

    # lists to store metrics for each fold
    fold_losses = []
    fold_precisions = []
    fold_recalls = []
    fold_f1_scores = []
    f1_scores_per_fold = []  # List to store F1 scores for each fold
    


    # Perform K-Fold Cross-Validation
    for fold, (train_idx, val_idx) in enumerate(kf.split(dataset)):
        print("-" * 50)
        print(f"------ Fold {fold + 1 }/{num_folds} ------")

        # Create data loaders for the train and validation sets
        train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
        val_sampler = torch.utils.data.SubsetRandomSampler(val_idx)
        
        train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler, collate_fn=collate_fn)
        val_dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=val_sampler, collate_fn=collate_fn)
        
        # Initialize the model, dense layer, criterion, and optimizer for each fold
        model = model_class(output_dim, output_dim, device).to(device)
        
        dense_layer = dense_layer_class(input_size=output_dim*2).to(device)
        criterion = BCELoss()
        optimizer = get_optimizer(list(model.parameters()) + list(dense_layer.parameters()), lr=learning_rate)

        # Training loop for each fold
        for epoch in range(num_epochs):
            print(f"Epoch {epoch + 1}/{num_epochs}")
            
            # Train and evaluate the model on the training and validation sets
            train_loss = train_model(model=model, dense_layer=dense_layer, dataloader=train_dataloader, criterion=criterion, optimizer=optimizer, device=device)
            val_loss, precision, recall, f1_score = evaluate_model(model=model, dense_layer=dense_layer, dataloader=val_dataloader, criterion=criterion, device=device)
            
            print(f"Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")
            print(f"Validation Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1_score:.4f}")
        
        # Store the validation metrics for this fold
        print(f"Results for Fold {fold}: Validation Loss = {val_loss:.4f}, Precision = {precision:.4f}, Recall = {recall:.4f}, F1 Score = {f1_score:.4f}")
        fold_losses.append(val_loss)
        fold_precisions.append(precision)
        fold_recalls.append(recall)
        fold_f1_scores.append(f1_score)
        f1_scores_per_fold.append(f1_score)  # Save F1 score for the current fold

    # Calculate the average metrics across all folds
    avg_loss = np.mean(fold_losses)
    avg_precision = np.mean(fold_precisions)
    avg_recall = np.mean(fold_recalls)
    avg_f1_score = np.mean(fold_f1_scores)

    print("-" * 50)
    print("\nK-Fold Cross-Validation Results:")
    print(f"Average Loss: {avg_loss:.4f}")
    print(f"Average Precision: {avg_precision:.4f}")
    print(f"Average Recall: {avg_recall:.4f}")
    print(f"Average F1 Score: {avg_f1_score:.4f}")

    # Save F1 scores per fold to a .npy file
    np.save(output_file, np.array(f1_scores_per_fold))
    print(f"F1 scores per fold saved to {output_file}")
    
    # Testing the model
    print("-" * 40)
    print("Testing the model on the test set...")
    test_scores = test_model(model=model, dense_layer=dense_layer, dataloader=test_dataloader, criterion=criterion, device=device)


    return avg_loss, avg_precision, avg_recall, avg_f1_score, test_scores

In [21]:
# DenseLayer or FinalClassifier
if isFinalClassifier:
    dense_layer_class = FinalClassifier
else:
    dense_layer_class = DenseLayer

# Run k-fold cross-validation   
results = cross_validate_model(
    dataset=full_dataset,
    model_class=SMCAModel,
    dense_layer_class=dense_layer_class,
    num_folds=5,
    num_epochs=num_epochs,
    batch_size=32,
    learning_rate=learning_rate,
    output_file="results/SMCA-F1_scores.npy"
)



Device: cuda
--------------------------------------------------
------ Fold 1/5 ------
Epoch 1/10
--------------- Eval ---------------
Movie Trailer IDs:
 ('tt0332658', 'tt0109444', 'tt0328880', 'tt1646971', 'tt2298990', 'tt1664894', 'tt0078788', 'tt0114279', 'tt0346156', 'tt1107365', 'tt0421073', 'tt1153706', 'tt1031280', 'tt2170299', 'tt0087469', 'tt5512872', 'tt0040662', 'tt0304711', 'tt0364045', 'tt0472399', 'tt0097722', 'tt0772157', 'tt10054316', 'tt0424095', 'tt0281322', 'tt0363988', 'tt0815241', 'tt0110950', 'tt1712261', 'tt0896866', 'tt0120913', 'tt0163187')
Predictions (raw):
  [0.575 0.57  0.575 0.566 0.572 0.568 0.57  0.563 0.572 0.579 0.572 0.578
 0.572 0.558 0.574 0.568 0.571 0.573 0.569 0.569 0.571 0.572 0.565 0.572
 0.571 0.565 0.572 0.573 0.572 0.568 0.565 0.563]
Binary Predictions:
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.]
Targets:
            [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1.

KeyboardInterrupt: 